In [1]:
import os
os.chdir("../../")

In [2]:
import sqlite3
import pandas as pd
from api.utils.database import rows_to_dicts

In [3]:
con = sqlite3.connect("./pipeline/database.db")

In [5]:
cur = con.cursor()
rows = cur.execute("""
SELECT *
FROM cta_train_stops
LIMIT 5
""").fetchall()

rows

pd.DataFrame(rows_to_dicts(cur, rows))

,stop_id,direction_id,stop_name,station_name,station_descriptive_name,map_id,ada,red,blue,g,brn,p,pexp,y,pnk,o,location
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,1,0,0,0,0,0,0,0,1,0,"{'latitude': '41.857908', 'longitude': '-87.66..."
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,1,0,0,0,0,0,0,0,1,0,"{'latitude': '41.857908', 'longitude': '-87.66..."
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,1,0,0,0,0,0,0,0,0,1,"{'latitude': '41.829353', 'longitude': '-87.68..."
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,1,0,0,0,0,0,0,0,0,1,"{'latitude': '41.829353', 'longitude': '-87.68..."
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,1,0,0,1,0,0,0,0,0,0,"{'latitude': '41.831677', 'longitude': '-87.62..."
